# Feature selection - SRANK
Nel notebook `SRANK_results.ipynb` abbiamo determinato la score di importanza per ogni feature, ora applichiamo questa conoscenza alla feature selection.
La logica è la seguente:
* Dato un algoritmo tipo k-Means, parto con la feature più importante e trovo il valore ottimale di cluster
* In seguito aggiungo la seconda feature e trovo il valore ottimale di cluster
* Così fino ad aver esaurito tutte le features
* Alla fine confronto i risultati ottenut i per il numero di feature migliore

In [1]:
#############################################
#   IMPORT                                  #
#############################################
#librerie
import pandas as pd 
import numpy as np 
import math
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
#dataset
df = pd.read_csv("../DatasetClientClustering.csv", 
                 engine='c', 
                 sep=',', 
                 encoding='latin-1')
df = df.drop(df.columns[list(np.arange(8))], axis=1)
df = df.drop(columns=["ClientID"])
#seleziono le feature che mi interessa clusterizzare
current_investment = ["PortfolioRisk", "PortfolioHorizon", 
                      "AuM", "BondInvestments", "EquityInvestments",
                      "MoneyMarketInvestments", "OtherInvestments", "Cash"]
person_var = ["RiskPropension", "ClientInvestmentHorizon", 
              "ClientKnowledgeExperience", "ClientPotentialIndex",
              "IncomeHighLow", "Sex", "Age", "IncomeNeed", 
              "LongTermCareNeed", "ProtectionNeed",
              "InheritanceIndex", "PanicMood", 
              "ClientDateStart", "NoTrustInBanks"]
#DATASET CON I PUNTI
dataframe = df[person_var]

#ora importo la tabella dei rankings
rankings = (pd.read_csv("SRANKresults.csv", sep = ";", index_col = [0])
             .sort_values(by = "score_final", ascending = False))

In [6]:
#############################################
#   K-MEANS - Calinski-Harabasz Index       #
#############################################
#si seleziona per ogni gruppo di feature il numero di cluster ottimale
#si confrontano poi i risultati ottimali per ogni gruppo di features

#array di features considerate per il clustering
feature_group = []

#tengo le statistiche nei seguenti 3 vettori
bestof_group_CHindex = []
bestof_group_ncluster = []
bestof_group_feature_group = [i for i in range(len(rankings.index))]

for feature in rankings["feature"]:
    #aggiungo una feature al gruppo e prendo i dati
    feature_group.append(feature)
    data_section = dataframe[feature_group]
    #stampo i progressi
    print("Feature group: ", len(feature_group), 
        " / ", len(rankings.index))
    
    #vario il numero di cluster fra 1 e 20
    #per ognuno calcolo l'indice 
    for nClusters in range(2, 21):
        #stampo i progressi
        print("NCluster: ", nClusters), 
        print("\r")
        
        #array dei valori del parametro
        nCluster_scores = []
        
        #fitto e calcolo, aggiungo il valore all'array
        kmeans_model = KMeans(n_clusters = nClusters, random_state = 1)
        kmeans_model.fit(data_section)
        labels = kmeans_model.labels_
        nCluster_scores.append(metrics.calinski_harabasz_score(data_section, labels))
    
    #ora salvo i valori migliori nei vettori
    best_CH = max(nCluster_scores)
    nClusters_best = nCluster_scores.index(best_CH) + 2
    bestof_group_CHindex.append(best_CH)
    bestof_group_ncluster.append(nClusters_best)

#creo un dataframe e plotto
results = pd.DataFrame({
                            "CH_index" : bestof_group_CHindex, 
                            "n_clusters" : bestof_group_ncluster, 
                            "feature_group" : bestof_group_feature_group
                        })



Feature group:  1  /  14
Feature group:  2  /  14
Feature group:  3  /  14
Feature group:  4  /  14
Feature group:  5  /  14
Feature group:  6  /  14
Feature group:  7  /  14
Feature group:  8  /  14
Feature group:  9  /  14
Feature group:  10  /  14
Feature group:  11  /  14
Feature group:  12  /  14
Feature group:  13  /  14
Feature group:  14  /  14
NCluster:  4NCluster:  5NCluster:  6NCluster:  7NCluster:  8NCluster:  9NCluster:  10NCluster:  11NCluster:  12NCluster:  13NCluster:  14NCluster:  15NCluster:  16NCluster:  17NCluster:  18NCluster:  19NCluster:  20

,CH_index,n_clusters,feature_group
0,46320.568407,2,0
1,46953.091262,2,1
2,5288.689276,2,2
3,3382.697205,2,3
4,1778.738150,2,4
5,1303.356266,2,5
6,1250.071904,2,6
7,1079.017499,2,7
8,1269.608224,2,8
9,1299.788671,2,9
